# Image Classification

First, set up your notebook as in previous scripts. 

In [1]:
# Install geemap library so that we can use it to view images on an interactive map
## You may get a runtime warning - you can ignore that
!pip install geemap
# We will need another package called geopandas if we are doing any projection of data
!pip install geopandas

     |████████████████████████████████| 471kB 2.8MB/s 
     |████████████████████████████████| 81kB 4.7MB/s 
     |████████████████████████████████| 1.3MB 5.3MB/s 
     |████████████████████████████████| 143kB 13.3MB/s 
     |████████████████████████████████| 225kB 13.3MB/s 
     |████████████████████████████████| 81kB 6.9MB/s 
     |████████████████████████████████| 3.3MB 13.4MB/s 
     |████████████████████████████████| 102kB 9.8MB/s 
     |████████████████████████████████| 1.6MB 33.4MB/s 
     |████████████████████████████████| 102kB 9.6MB/s 
     |████████████████████████████████| 1.2MB 35.7MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 552kB 38.1MB/s 
     |████████████████████████████████| 122kB 37.8MB/s 
     |████████████████████████████████| 389kB 31.2MB/s 
     |████████████████████████████████| 122kB 37.6MB/s 
     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
  Crea

     |████████████████████████████████| 1.0MB 4.1MB/s 
     |████████████████████████████████| 15.4MB 195kB/s 
     |████████████████████████████████| 6.6MB 36.3MB/s 


In [3]:
# Import the necessary libraries
import ee
import numpy as np
import geemap.eefolium as geemap
import geopandas as geopandas
import json
import pandas as pd

In [4]:
# Authenticate and initialize this instance of GEE in Google Colab
## Follow the prompts and fill in authentication code
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=hWu90c7sJW3uAr9qlAqYW7p4D1Uus8lEsZfynZZmVLU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7eirLYAjpfSzFe2wwhsAD8Creqc-uH2HoD9X5Rl6a7VnEUn6F-HFA

Successfully saved authorization token.


# Import some imagery.

In [7]:
# Import the landsat-8 tier 1 surface reflectance as a variable
l8SR = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') 

# Create a point at Alfred Bog (east of Ottawa)
point = ee.Geometry.Point([-74.8588,45.4839]) 

# Filter for images within our date range that overlap our point
# Sort by cloud cover and choose least cloudy (first)
l8SR_Jul2019_LC = l8SR.filterBounds(point) \
                      .filterDate('2020-06-01', '2020-08-31') \
                      .sort("CLOUD_COVER") \
                      .first()

#Import our training data from Google Drive
Since we are doing supervised image classification, let's import some "training data".

We will visualize the training data (coloured by class) and add the imagery as well for comparison.

In [8]:
Map = geemap.Map()

# Mounting your google drive
from google.colab import drive
drive.mount('/content/drive')

# this dataset should only be used for education purposes.  For any other uses, please request permission from Koreen Millard. 
AlfredPts = '/content/drive/MyDrive/CSRS2021_GEEWorkshop_ForStudents/datasets/training_dataAlfred.shp'

shpdata = geemap.shp_to_ee(AlfredPts)

L8_TC_vis = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}


vis_params = {
    'color': '000000', 
    'colorOpacity': 1,
    'pointSize': 3,
    'pointShape': 'circle',
    'width': 1,
    'lineType': 'solid', 
    'fillColorOpacity': 1    
}

palette = ['D6E51B', '288857', '461BE5']  # We have 3 classes so need 3 colours in our pallette

Map.addLayer(l8SR_Jul2019_LC,L8_TC_vis,'l8SR_Jul2019_LC')
Map.add_styled_vector(shpdata, column="ClassID", palette=palette, layer_name="Styled vector", **vis_params)  # ClassID is the field that we are using to colour our points

Map.addLayerControl()
Map.centerObject(shpdata,10)

Map

Mounted at /content/drive


Examine our training data 

In [ ]:
# we will convert our ee features to a pandas data frame
df = geemap.ee_to_pandas(shpdata)
df.head() #the head function just displays the first few records 

# Perform Supervised Image Classification

Now, let's set up the image classification.
First we extract the values in each of the image bands for each point.

In [9]:
# Make a list of the names of the bands you want to extract (you can pick and choose)
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11'] # Specify bands to be used in training data
# The class info is stored in the 'ClassID' column
label = 'ClassID'

# Extract pixel values to training data points
training = l8SR_Jul2019_LC.select(bands).sampleRegions(
  collection = shpdata,
  properties = [label],
  scale = 30
)


# To examine our training data, we will convert our ee features to a pandas data frame
df = geemap.ee_to_pandas(training)
df.head() #the head function just displays the first few records 

,B10,B11,B2,B3,B4,B5,B6,B7,ClassID
0,3093,3054,306,401,567,1543,2137,1788,1
1,3070,3040,319,404,534,1540,3544,2688,1
2,3026,3005,461,604,601,2211,2750,2058,1
3,3064,3028,780,1080,1396,2293,3440,2647,1
4,2966,2950,266,442,250,4801,1966,803,1


Now, we will run the random Forest classifier.

In [10]:
# First we train a model
trained = ee.Classifier.smileRandomForest(250).train(training, label, bands)

# Next we run the model on our image to produce a new variable that contains the classified "map"
classified = l8SR_Jul2019_LC.select(bands).classify(trained) # Run the classifier

Now we will visualize the result.

In [11]:
Map = geemap.Map()


Map.addLayer(l8SR_Jul2019_LC,L8_TC_vis,'l8SR_Jul2019_LC')

palette = ['D6E51B', '288857', '461BE5']  # We have 3 classes so need 3 colours in our pallette
vis_params = {
  'min': 0,
  'max': 3,
  'palette': palette}

Map.addLayer(classified, vis_params, 'Classified Result')

Map.addLayerControl()
Map.centerObject(shpdata,10)

Map

# Export the results to Google Drive

In [12]:
geemap.ee_export_image_to_drive(classified, description='ClassifiedImageOutput_geemap', folder='CSRS2021_GEEWorkshop_ForStudents', scale=30)

Exporting ClassifiedImageOutput_geemap ...
